# **Install dependencies**

In [ ]:
!pip install -q ultralytics opencv-python-headless numpy tqdm

import cv2
import numpy as np
from ultralytics import YOLO
from google.colab import files
import os
from IPython.display import display, HTML
from tqdm.notebook import tqdm
import time

# **Load YOLOv8 object detection model with GPU support.**

In [ ]:
def load_model():
    return YOLO("yolov8x.pt")  # Large model for high accuracy

# **Define ROI covering the upper part of the escalator.**

In [ ]:
def define_roi(width, height):
    return [
        int(width * 0.45),  # x1: left edge
        int(height * 0.15), # y1: top edge
        int(width * 0.65),  # x2: right edge
        int(height * 0.40)  # y2: bottom edge (upper part of escalator)
    ]

# **Check if a bounding box's center is inside the ROI.**

In [ ]:
def is_in_roi(box, roi):
    x1, y1, x2, y2 = box
    center_x = (x1 + x2) / 2
    center_y = (y1 + y2) / 2
    rx1, ry1, rx2, ry2 = roi
    return rx1 <= center_x <= rx2 and ry1 <= center_y <= ry2

# **Determine ROI rectangle color and message based on person count.**

In [ ]:
def get_roi_color_and_message(count):
    if count == 2:
        return (0, 255, 0), "Normal"  # Green for 2 people
    elif count == 3:
        return (0, 255, 255), "Care"  # Yellow for 4 people
    elif count > 3:
        return (0, 0, 255), "Warning"  # Red for >4 people
    return (255, 255, 255), ""  # Default white, no message for other counts

# **Process video with tracking, apply colored ROI rectangle with message, and save output.**

In [ ]:
def process_video(video_path, output_path):
    # Load model and video
    model = load_model()
    cap = cv2.VideoCapture(video_path)
    if not cap.isOpened():
        print(f"Error: Could not open video {video_path}")
        return False

    # Video properties
    fps = int(cap.get(cv2.CAP_PROP_FPS))
    width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    frame_count = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    roi = define_roi(width, height)

    # Initialize video writer
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    out = cv2.VideoWriter(output_path, fourcc, fps, (width, height))
    if not out.isOpened():
        print("Error: Could not initialize video writer")
        cap.release()
        return False

    # Process frames with progress bar
    print(f"Processing video with {frame_count} frames...")
    start_time = time.time()
    progress_bar = tqdm(total=frame_count, desc="Processing Frames")

    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break

        # YOLOv8 prediction with tracking (only detect people, class 0)
        results = model.track(frame, stream=True, classes=[0], device='cuda', conf=0.5, iou=0.7, persist=True)

        person_count_in_roi = 0
        annotated_frame = frame.copy()

        for result in results:
            boxes = result.boxes.xyxy.cpu().numpy()
            track_ids = result.boxes.id.cpu().numpy() if result.boxes.id is not None else []

            # Count unique people in ROI using track IDs
            unique_ids = set()
            for i, box in enumerate(boxes):
                if is_in_roi(box, roi) and i < len(track_ids):
                    unique_ids.add(track_ids[i])
            person_count_in_roi = len(unique_ids)

            # Draw green bounding boxes for people in ROI
            for box in boxes:
                if is_in_roi(box, roi):
                    x1, y1, x2, y2 = map(int, box)
                    cv2.rectangle(annotated_frame, (x1, y1), (x2, y2), (0, 255, 0), 2)

        # Get ROI color and message based on count
        roi_color, message = get_roi_color_and_message(person_count_in_roi)

        # Draw ROI rectangle with conditional color
        cv2.rectangle(annotated_frame, (roi[0], roi[1]), (roi[2], roi[3]), roi_color, 2)

        # Add message on ROI rectangle (centered)
        if message:
            text_size = cv2.getTextSize(message, cv2.FONT_HERSHEY_SIMPLEX, 0.7, 2)[0]
            text_x = roi[0] + (roi[2] - roi[0] - text_size[0]) // 2
            text_y = roi[1] + (roi[3] - roi[1] + text_size[1]) // 2
            cv2.putText(annotated_frame, message, (text_x, text_y), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255, 255, 255), 2)

        # Add crowd density text at top-left
        text = f"Crowd Density: {person_count_in_roi}"
        cv2.putText(annotated_frame, text, (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2)

        # Write frame to output
        out.write(annotated_frame)
        progress_bar.update(1)

    # Clean up
    progress_bar.close()
    cap.release()
    out.release()
    elapsed_time = time.time() - start_time
    print(f"Processing complete! Total time: {elapsed_time:.2f} seconds")
    return True

# **Main function to handle video upload, processing, and auto-download.**

In [ ]:
def main():
    # Upload video
    print("Please upload your video file (e.g., input_video.mp4)")
    uploaded = files.upload()
    if not uploaded:
        print("Error: No video uploaded.")
        return

    video_path = list(uploaded.keys())[0]
    output_path = "output_density_mall.mp4"

    # Process video
    if process_video(video_path, output_path):
        # Auto-download output video
        if os.path.exists(output_path):
            print("Initiating automatic download of processed video...")
            files.download(output_path)
        else:
            print("Error: Output video was not created.")

if __name__ == "__main__":
    main()